In [5]:
from pymongo import MongoClient

In [6]:
client = MongoClient('localhost', 27017)
db = client['local']  # Nom de la base de données
users_collection = db['Users']  # Nom de la collection

# Lire tous les documents dans la collection 'Users'
all_users = list(users_collection.find())  # Convertir le curseur MongoDB en liste



In [7]:
all_users 

[{'_id': ObjectId('67082846a5277fd2cd339d86'),
  'First Name': 'Lucas',
  'Last Name': 'Rey',
  'Profession': 'Engineer',
  'Age': 51,
  'Gender': 'Non-binary',
  'Address': '50 Rue des Peupliers, 92290 Châtenay-Malabry, France',
  'Latitude': 48.757015,
  'Longitude': 2.267142,
  'Service Tag': 'Prêt de matériel de bricolage',
  'Email': 'lucasrey@gmail.com',
  'Password': '1234',
  'Status': 'GenDemandeur',
  'Demands': [{'numero': 1,
    'tag': 'Prêt de matériel de bricolage',
    'description': 'Service de Prêt de matériel de bricolage',
    'date': '2023-01-01',
    'statut': 'actif'},
   {'numero': 2,
    'tag': 'Prêt de matériel de bricolage',
    'description': 'Service de Prêt de matériel de bricolage',
    'date': '2023-08-01',
    'statut': 'actif'}],
  'Commentaires': 'Produit à éviter, mauvaise qualité.',
  'Points': 368},
 {'_id': ObjectId('67082846a5277fd2cd339d87'),
  'First Name': 'Martin',
  'Last Name': 'Baron',
  'Profession': 'Lawyer',
  'Age': 54,
  'Gender': 'Non

In [3]:

import pandas as pd
import numpy as np
import json
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans
import openpyxl

# Télécharger les ressources NLTK
nltk.download('vader_lexicon')

client = MongoClient('localhost', 27017)
db = client['local']  # Nom de la base de données
users_collection = db['Users']  # Nom de la collection

# Lire tous les documents dans la collection 'Users'
all_users = list(users_collection.find())  # Convertir le curseur MongoDB en liste
df = pd.json_normalize(all_users, record_path=['Demands'])


# Analyse des sentiments
sia = SentimentIntensityAnalyzer()

def analyze_sentiment(comments):
    sentiments = []
    for comment in comments:
        score = sia.polarity_scores(comment)
        sentiments.append(score['compound'])
    return np.mean(sentiments)



def sentiment_label(score):
    if score > 0.05:
        return 'Positif'
    elif score < -0.05:
        return 'Négatif'
    else:
        return 'Neutre'
df['Average Sentiment'] = df['Commentaires'].apply(analyze_sentiment)
df['Sentiment Label'] = df['Average Sentiment'].apply(sentiment_label)

df_sentiments = df[['First Name', 'Last Name', 'Commentaires', 'Average Sentiment', 'Sentiment Label']]
df_sentiments.to_excel('sentiment_analysis_results.xlsx', index=False)

# Classification
features = df[['Age', 'Gender', 'Points', 'Average Sentiment']]
le_gender = LabelEncoder()
features['Gender'] = le_gender.fit_transform(features['Gender'])
features.fillna(features.mean(), inplace=True)
target = df['Status']
le_status = LabelEncoder()
target_encoded = le_status.fit_transform(target)
X_train, X_test, y_train, y_test = train_test_split(features, target_encoded, test_size=0.2, random_state=42)
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
df_classification = X_test.copy()
df_classification['Actual Status'] = le_status.inverse_transform(y_test)
df_classification['Predicted Status'] = le_status.inverse_transform(y_pred)
df_classification.to_excel('classification_results.xlsx', index=False)

# Clustering
clustering_features = df[['Age', 'Points', 'Average Sentiment']]
scaler = StandardScaler()
clustering_features_scaled = scaler.fit_transform(clustering_features)
kmeans = KMeans(n_clusters=3, random_state=42)
df['Cluster'] = kmeans.fit_predict(clustering_features_scaled)
df_clustering = df[['First Name', 'Last Name', 'Age', 'Points', 'Average Sentiment', 'Cluster']]
df_clustering.to_excel('clustering_results.xlsx', index=False)


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Meriem\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


KeyError: 'Commentaires'

In [4]:
df

NameError: name 'df' is not defined